In [28]:
%matplotlib inline

import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import string
import collections

In [18]:
df = pd.read_csv("../data/scrubbed.csv")
c_df = df
df.head()

/Users/tomi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [19]:
# Object column into string
c_df['comments'] = c_df['comments'].astype(str)

# All to lower caps
c_df.comments = c_df.comments.apply(lambda x: x.lower())

# Remove punctuation
c_df['comments'] = c_df['comments'].str.replace('[^\w\s]','')

c_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,this event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 lackland afb44 tx lights racing across t...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,greenorange circular disc over chester44 england,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,my older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,as a marine 1st lt flying an fj4b fighterattac...,1/22/2004,21.4180556,-157.803611


In [20]:
#Remove stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
c_df['comments'] = c_df['comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

/Users/tomi/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


[nltk_data] Downloading package stopwords to /Users/tomi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
#Stemmer
from __future__ import print_function
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
c_df['comments'] = c_df['comments'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

c_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,event took place earli fall around 194950 occu...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 lackland afb44 tx light race across sky a...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,greenorang circular disc chester44 england,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,older brother twin sister leav edna theater 9 ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,marin 1st lt fli fj4b fighterattack aircraft s...,1/22/2004,21.4180556,-157.803611


In [26]:
# write to text file
file_comments = open("../data/ufo_comments_descriptions.txt", "w")
for row in c_df.itertuples(index=True, name='Pandas'):
    val = getattr(row, "comments")
    file_comments.write(val)
file_comments.close()

Read from text file that was made above, and do stuff

In [35]:
# Most common words
comments_df = pd.read_table("../data/ufo_comments_descriptions.txt", header=None)
comments_df.columns = ["comments"]

comments_most_common = collections.Counter(" ".join(comments_df["comments"].dropna()).split()).most_common(10)
print("Most common words in comments/descriptions:")
print(comments_most_common)

Most common words in comments/descriptions:
[('light', 29268), ('object', 15049), ('move', 11702), ('sky', 10543), ('shape', 8360), ('bright', 7577), ('white', 5951), ('orang', 5603), ('seen', 5128), ('fli', 5092)]


In [41]:
#tf/idf
from sklearn.feature_extraction.text import TfidfVectorizer

# Ignore too common terms (max_df = .25)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(comments_df['comments'])
print(vectorizer.get_feature_names())

['00', '000', '0005hrs', '0006', '000mphgiant', '0010bodi', '0014hrs', '0015', '0015am44', '0015hour', '0020', '0023second', '0025', '0030', '0040', '0043', '0045unexplain', '0046am', '00472', '0055', '0056', '006', '007quick', '00adult', '01', '0100', '0100hour', '010103', '010109red', '01011979', '0103', '010307', '0104043', '010411', '01042006realli', '010507', '010599', '010612red', '01062013', '010712010', '01092010', '011027saw', '011104', '011109two', '011112small', '01132012ufo', '0115', '01152012', '011600', '011608odd', '01162010white', '01162013unidentifi', '011709two', '01180111081001', '01182010', '011995', '012009', '01212002', '012210triangl', '012302', '01242012five', '0125090100', '012556', '012609', '012811', '013', '0130', '0130am', '01312012', '0144', '01442008', '0145', '0145am44', '0145amsilent', '01749', '01over', '02', '020', '0200', '0200030044', '0200singl', '020202', '02022010februari', '02022014low', '02032013fli', '020708', '020808came', '020808took', '0210

In [42]:
# sort by means of words
comments_highest =  [vectorizer.get_feature_names()[id] for id in (-X.mean(axis=0).A).argsort()[0][:10]]
print("Comments highest score")
print(comments_highest)

Comments highest score
['light', 'object', 'move', 'sky', 'shape', 'bright', 'white', 'orang', 'seen', 'fli']


In [43]:
# Plotting
from wordcloud import WordCloud

ModuleNotFoundError: No module named 'wordcloud'